**Work completed:** Implemented missing functions in `Reg.py` and `auxiliary_func.py` (proximal-gradient solvers, Huber gradient/loss, PCR/PLS, OLS/OLSH, PCAR/PLSR, oracle regression, and VIP utilities).

# Part 0: Set up environment

Run the following import statements and ensure there are no errors before continuing.
If you are missing packages, use `! pip install %` to install package `%` as below.

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install scipy
# !pip install sklearn
# !pip install ipykernel

In [ ]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

## Generate firm-characteristics and returns 
Run the following cell to generate the dataset. After running it, you should see a new folder called `Simu` and one subfolder called `SimuData_` plus **a data tag**, i.e. `SimuData_100`. In this folder, you will get the charactistics and returns of the linear and nonlinear models. 

Remember you don't need to change anything in `DGP.py`, but make sure to read through it and understand what each csv file represents. You should change the data number parameter *P* and horizon parameter *hh* which allows you to have different return resolutions ie monthly, quarterly, etc.

In [ ]:
from DGP import dgp
P     =   100
hh    =   [1,3]
dgp(P,hh)

# Part 1: Preparation functions
In this section, you will first implement some preparation functions for creating the models. For example, to update the coefficient matrix, we should do some gradient descent on the original matrix, so we introduce Accelarated Proximal Gradient Algorithm to update the matrix. 

This section is cited of paper [Empirical Asset Pricing via Machine Learning](https://dachxiu.chicagobooth.edu/download/ML.pdf).

Now, open the file `auxiliary_func.py`. In this file, we have already provided several functions that will be used later. Read through the functions: `fw2()`, `fw1()` and `sq()`. Run the next cell to show what each function really does.

In [ ]:
from auxiliary_func import fw2,fw1

In [ ]:
x = np.array([[1,3,2],[9,3,6],[2,4,6]])
index = fw2(x)
print(index)      # you should get the result of [1,0]

In [ ]:
x = np.array([1,7,4,3,5])
index = fw1(x)
print(index)    # you should get the reuslt of 1

The function `sq(a,b,step)` should have the same functionality as `np.arange(a,b+1e-10,step)`. Note that we add a small value to b since np.arange() will not include the end in the array, but our function should include it. 

In [ ]:
from auxiliary_func import sq
a = 1.0
b = 5.0
step = 1
x = sq(a,b,step)
print(x)

### Generalized linear
Linear models are popular in practice, however, the “true” model is sometimes complex and nonlinear, so if we restrict the functional form to be linear it may introduce approximation error due to model misspecification. Thus, we introduce a nonparametric approach: the generalized linear model. You can find more details in Section 1.5 in paper [Empirical Asset Pricing via Machine Learning](https://dachxiu.chicagobooth.edu/download/ML.pdf).

In the generalized linear model, we implement a vector of basic functions. To simplify the function, we first compute these basic functions. Open the file `Reg.py`.  Follow the function description in the paper to implement the function `cut_knots_degree2(x,n,th)` and test your code. You should get error of magnitude `e-4` or less.

In [ ]:
from Reg import cut_knots_degree2
N = 3
n = 4
T = 2

x      = np.linspace(-0.2, 0.4, num=N*T).reshape(N, T)
th     = np.linspace(0.3,1.6,num=n*T).reshape(n,T)
result = cut_knots_degree2(x,n,th)
expect_result = np.array([
    [-0.24,0.144,0,0,0,-0.24,0.175542857,0,0,0],
    [0,-0.0384,0,0,0,5.55111512e-17,-0.0384,0,0,0],
    [0.24,-0.1056,0,0,0,0.24,-0.13712857,0,0,0]
])
print(rel_error(result,expect_result))

## Ordinary Least Square 
We begin our model practice with the least complex method, the simple linear predictive regression model estimated via ordinary least squares (OLS). Although we expect this to perform poorly in our high dimensional problem, we use it as a benchmark to emphasize the efficacy of more complex models.
Our baseline estimation of the simple linear model uses a standard least squares, or “*l2*” objective function:

$$L(\theta) = \frac{1}{NT} \sum_{i=1}^{N} \sum_{t=1}^{T}
(r_{i,t+1}-g(z_{i,t};\theta))^{2}$$

Open file `Reg.py` and implement the function `loss(y,yhat)`. Test your code here and you should see errors of magnitude `e-9` or less.

In [ ]:
from Reg import loss
y           = np.linspace(-0.2, 0.4, num = N*T)
yhat        = np.linspace(-0.1,0.5, num = N*T)
diff        = loss(y,yhat)
expect_diff = 0.0099999999
print(rel_error(diff,expect_diff))

## Ordinary Least Square and Huber Function
Heavy tails are a well-known attribute of financial returns and stock-level predictor variables. Convexity of the least squares loss places extreme emphasis on large errors, thus outliers can undermine the stablity of OLS-based prediction.

In the machine learning literature, a common choice for counteracting the deleterious effect of heavy-tailed observations is the Huber robust objective function:
$$L_{H}(\theta)=\frac{1}{NT}\sum_{i=1}^{N}\sum_{t=1}^{T}
H\left(r_{i,t+1}-g(z_{i,t};\theta),\xi\right)$$


$$H(x;\xi)=
\begin{cases}x^2, &if\quad|x|\leq\xi, \\
2\xi|x|-\xi^2, &if\quad|x|>\xi
\end{cases}
$$

Open file `Reg.py` and implement the function `lossh(y,yhat,mu)`, test your code here and you should see errors of magnitude`e-16` or less.

In [ ]:
from Reg import lossh
y    = np.linspace(-0.2, 0.4, num = N*T)
yhat = np.linspace(-0.1,0.5, num = N*T)
mu   = 0.001
diff = lossh(y,yhat,mu)
expect_diff = 0.000199
print(rel_error(diff,expect_diff))

**Extension**: Robust objective functions. In some cases, replacing oridinary squared loss equation with a weighted least squares objective, such as

$$\mathcal{L}_{\omega}(\theta) = \frac{1}{NT} \sum_{i=1}^{N} \sum_{t=1}^{T}
\omega_{i,t} \; (r_{i,t+1}\;) - g(z_{i,t};\theta))^{2}$$

can possibly improve the predictive performance. This weighted least squares
objective allows the econometrician to tilt estimates toward observations that
are more statistically or economically informative.

Open the file `Reg.py` and implement the functions `lossw()` and `losshw()`. These two functions should be very similar to `loss()` and `lossh()`. Test your code here, you should get errors of magnitude `e-10` or less.

In [ ]:
from Reg import lossw,losshw
np.random.seed(1)
y    = np.random.randn(10)
yhat = np.random.randn(10)
w    = np.random.randn(10)
mu   = 0.1
lw   = lossw(y,yhat,w)
expect_lw  = 1.85130397827934
lhw  = losshw(y,yhat,w,mu)
expect_lhw = 0.32452079903016
print(rel_error(lw,expect_lw))
print(rel_error(lhw,expect_lhw))

Still in `Reg.py`, implement the function `f_grad()` and `f_gradh()`, test your code here. You should see errors on the order of `e-8` or less.

In [ ]:
from Reg import f_grad
np.random.seed(1)
X    = np.random.randn(3,5)
Y    = np.random.randn(3,1)
w    = np.random.randn(5,1)
pred = f_grad(X,Y,w)
expect_pred = np.array([[0.94522685],[2.07035499],[0.67631656]])
print(rel_error(pred,expect_pred))

In [ ]:
from Reg import f_gradh
mu          = 1.0
w_grad      = f_gradh(w,X,Y,mu)
expect_grad = np.array([[0.22268398],[-0.22674411],[-1.47850512],[-0.95490198],[1.33542321]])
print(rel_error(w_grad,expect_grad))

Open the file `Reg.py` and read through the following functions: `soft_thresholdl()`, `soft_thresholdr()`, `soft_thresholde()`, `soft_thresholda()`, `soft_thresholdg()`. These functions are differet kinds of soft thresold functions for various models. Make sure you understand what each function really does, keeping in mind we'll use them later. You can also test their outputs in the following cells.  

In [ ]:
from Reg import soft_thresholdl,soft_thresholdr,soft_thresholde,soft_thresholda,soft_thresholdg
np.random.seed(3)
w  = np.random.randn(8,1) * 5
mu = 5
print(w)

In [ ]:
print("Soft_thresholdl:")
print(soft_thresholdl(w,mu))

In [ ]:
print("Soft_thresholdr:")
print(soft_thresholdr(w,mu))

In [ ]:
print("Soft_thresholde:")
print(soft_thresholde(w,mu))

In [ ]:
alpha = 0.8
print("Soft_thresholda:")
print(soft_thresholda(w,alpha,mu))

In [ ]:
print("Soft_thresholdg:")
print(soft_thresholdg(w,mu))

## Accelarated Proximal Gradient Algorithm

Open the file `Reg.py` and implement the function `proximal()` and `proximalH()` which will output the coefficient matrix for the simple linear regression and the regression with huber function separately. You should see an error of magnitude `e-6` or less.

In [ ]:
from Reg import proximal,soft_thresholdl
np.random.seed(2)
XX  = np.random.randn(6,6) * 0.01
XY  = np.random.randn(6,1) * 0.01
tol = 1e-5
L   = 10000
l1  = 1e-4
w   = proximal(XX,XY,tol,L,l1,soft_thresholdl)
expect_w = np.array([[0.02564119,-0.0468463,-0.00101769,
                      0.02964973,-0.01591977,0.04160531]])
print(rel_error(w,expect_w))

In [ ]:
from Reg import proximalH,soft_thresholdr
np.random.seed(3)
X   = np.random.randn(6,5) * 0.01
y   = np.random.randn(6,1) * 0.01
w   = np.random.randn(5) * 0.01
tol = 1e-5
L   = 10000
l1  = 1e-4
mu  = 0.001
wh  = proximalH(w,X,y,mu,tol,L,l1,soft_thresholdr)
expect_wh = np.array([-0.00923792,-0.01023876,0.01123978,-0.00131915,-0.01623285])

print(rel_error(wh,expect_wh))


## PCR and PLS
Penalized linear models use shrinkage and variable selection to manage high dimensionality by forcing the coefficients on most regressors near or exactly to zero. This can produce suboptimal forecasts when predictors are highly correlated.

The idea of predictor averaging, as opposed to predictor selection, is the essence of dimension reduction. Forming linear combinations of predictors helps reduce noise to better isolate the signal in predictors and helps decorrelate otherwise highly dependent predictors. Two classic dimension reduction techniques are principal components regression (PCR) and partial least squares (PLS).

PCR consists of a two-step procedure. In the first step, principal components analysis (PCA) combines regressors into a small set of linear combinations that best preserve the covariance structure among the predictors. In the second step, a few leading components are used in standard predictive regression. That is, PCR regularizes the prediction problem by zeroing out coefficients on low variance components.

Partial Least Squares performs dimension reduction by directly exploiting covariation of predictors with the forecast target. PLS regression proceeds as follows. For each predictor $j$ , estimate its univariate return prediction coefficient via OLS. This coefficient, denoted $\varphi_{j}$ , reflects the “partial” sensitivity of returns to each predictor $j$ . Next, average all predictors into a single aggregate component with weights proportional to $\varphi_{j}$ , placing the highest weight on the strongest univariate predictors, and the least weight on the weakest. In this way, PLS performs its dimension reduction with the ultimate forecasting objective in mind. To form more than one predictive component, the target and all predictors are orthogonalized with respect to previously constructed components, and the above procedure is repeated on the orthogonalized data set. This is iterated until the desired number of PLS components is reached.

You can find more details in Section 1.4 in paper [Empirical Asset Pricing via Machine Learning](https://dachxiu.chicagobooth.edu/download/ML.pdf).

Open the file `Reg.py` and implement the functions `PCR()` and `PLS()`. Check your code here, you should get an error of magnitude `e-8` or less.

In [ ]:
from Reg import PCR
N = 6
p = 4
A = 3
np.random.seed(3)
X = np.random.randn(N,p)
y = np.random.randn(N,1)
B = PCR(X,y,A)
expect_B = np.array([[0,0.5751487,0.46676701],
                    [0,-0.28096611,-0.36453058],
                    [0,0.37704483,0.40708078],
                    [0,0.44675735,0.50838344]])
print(rel_error(B,expect_B))

In [ ]:
from Reg import pls
B_pls        = pls(X,y,A)
expect_B_pls = np.array([[0,-0.12371227,-0.07619591],
                        [0,0.05231579,0.09294739],
                        [0,-0.21909422,-0.22530672],
                        [0,0.41948184,0.43975808]])
print(rel_error(B_pls,expect_B_pls))

## Variable Importance
After we generate the coefficient matrix, we want to know which coefficients are important, meaning the coefficient has a strong importance on return prediction. We define importance as the difference of in-sample $R^{2}$. Open `auxiliary_func.py` and implement function `vip()`, test your code here. You should get an error of magnitude `e-7` or less.

In [ ]:
from auxiliary_func import vip
np.random.seed(4)
N = 10
p = 8
x = np.random.randn(N,p) * 0.01
y = np.random.randn(N,1) * 0.01
m = np.mean(y) 
y = y - m    # demeaned
b = np.random.randn(p,1)
v = vip(b,x,y,m)
expect_v = np.array([[0.00602217,0.04295142,-0.1589286,-0.01933711, 
                      -0.94963987,0.3282685,-0.13133609,-0.09904148]])
print(rel_error(v,expect_v))


# Part 2: Simulation Regression Models
We follow the most common approach in the literature and select tuning parameters adaptively from the data in a validation sample. In particular, we divide our sample into three disjoint time periods that maintain the temporal ordering of the data. The first, or “training,” subsample is used to estimate the model subject to a specific set of tuning parameter values.

The second, or “validation,” sample is used for tuning the hyperparameters. We construct forecasts for data points in the validation sample based on the estimated model from the training sample. Next, we calculate the objective function based on forecast errors from the validation sample, and iteratively search for hyperparameters that optimize the validation objective (at each step reestimating the model from the training data subject to the prevailing hyperparameter values).

Tuning parameters are chosen from the validation sample taking into account estimated parameters, but the parameters are estimated from the training data alone. The idea of validation is to simulate an out-of-sample test of the model. Hyperparameter tuning amounts to searching for a degree of model complexity that tends to produce reliable out-of-sample performance. The validation sample fits are of course not truly out of sample, because they are used for tuning, which is in turn an input to the estimation. Thus, the third, or “testing,” subsample, which is used for neither estimation nor tuning, is truly out of sample and thus is used to evaluate a method’s predictive performance.

In this section, you will learn some regression models and try to implement them. Here's the regression models you will be familiar with: **OLS, OLS+H, PCR, PLS, Lasso, Lasso+H, Ridge, Ridge+H, ENet, ENet+H and Group Lasso, Group Lasso+H**, also including the **Oracle Regression Model**.


This section is cited of paper [Empirical Asset Pricing via Machine Learning](https://dachxiu.chicagobooth.edu/download/ML.pdf).

## OLS

Now, we can create the actual model to approximate returns of characteristics. First, the simpliest model: OLS, short for ordinary least square. The simple linear model assumes that conditional expectation $g^{\star}(·)$ can be approximated by a linear function of the raw predictor variables and the
parameter vector, $\theta$,
$$g(z_{i,t};\theta)=z_{i,t}'\;\theta$$

This model imposes a simple regression specification and does not allow for nonlinear effects or interactions between predictors.

Open the file `Reg.py` and implement the function `OLS()`. Test your code here and make sure your error is of magnitude `e-7` or less.

In [ ]:
from Reg import OLS
np.random.seed(5)
N      = 10
p      = 8
xtrain = np.random.randn(N,p) * 0.01
ytrain = np.random.randn(N,1) * 0.01
mtrain = np.mean(ytrain)

ytrain_demean    = ytrain - mtrain
xoos   = np.random.randn(N,p) * 0.01
yoos   = np.random.randn(N,1) * 0.01
r2_oos,r2_is,b,v = OLS(xtrain,ytrain,ytrain_demean,mtrain,xoos,yoos)
expect_roos = -2.62487715
expect_ris  = 0.93824603
expect_b    = np.array([[1.03246781,-0.57572949,0.30877116,2.22773706,
                         0.53579763,0.34007811,-0.01709376,-0.47917419]])
expect_v    = np.array([[8.41495322e-01,3.96755096e-01,1.74216766e-01,1.75101820e+00,
                         4.26636411e-01,1.30379085e-01,3.80488058e-04,2.41441866e-01]])
print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))

## OLS+H
After implementsing OLS, we can perform a simple prediction on training set. However, the convexity of the least squares loss places extreme emphasis on large errors, thus outliers can undermine the stablity of OLS-based prediction. The statistics literature, long aware of this problem, has developed modified least squares objective functions that tend to produce more stable forecasts than OLS in the presence of extreme observations. In the machine learning literature, a common choice for counteracting the deleterious effect of heavy-tailed observations is the Huber robust objective function. Open the file `Reg.py` and implement the function `OLSH()`. Test your code here and make sure your error is of magnitude `e-7` or less.

In [ ]:
from Reg import OLSH,soft_thresholdl
np.random.seed(6)
N      = 10
p      = 8
xtrain = np.random.randn(N,p) * 0.01
ytrain = np.random.randn(N,1) * 0.01
mtrain = np.mean(ytrain)

ytrain_demean = ytrain - mtrain
xoos   = np.random.randn(N,p) * 0.01
yoos   = np.random.randn(N,1) * 0.01
mu     = 0.001
tol    = 1e-5
L      = 10000
r2_oos,r2_is,b,v = OLSH(xtrain,ytrain,ytrain_demean,mtrain,xoos,yoos,\
                        mu,tol,L,soft_thresholdl)
expect_roos = -10.46818178
expect_ris  = 0.85998362
expect_b    = np.array([[-1.10864565,-0.33660359,1.30256235,0.23021716,
                         -0.00943861,-0.96309068,1.00235173,-0.20870945]])
expect_v    = np.array([[1.41248027e+00,1.00154851e-01,1.97150088e+00,3.70812396e-02,
                         1.46988180e-04,1.09638383e+00,6.60536012e-01,5.29987272e-02]])
print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))

## PCA Regression and PLS Regrassion

Our implementation of PCR and PLS begins from the vectorized version of
the linear model. In particular, we reorganize the linear regression $r_{i,t+1} = z_{i,t}' \theta + \epsilon_{i,t+1}$ as
$$ R = Z\theta + E$$

where $R$ is the $NT \times 1$ vector of $r_{i,t+1}$, $Z$ is the $NT \times P$ matrix of stacked predictors $z_{i,t}$, and $E$ is a $NT \times 1$ vector of residuals $\epsilon_{i,t+1}$.

PCR and PLS take the same general approach to reducing the dimensionality. They both condense the set of predictors from dimension $P$ to a much smaller number of $K$ linear combinations of predictors. Thus, the forecasting model for both methods is written as

$$R = (Z \Omega_{K}) \theta_{K} + \tilde{E}$$

$\Omega_{K}$ is $P \times K$ matrix with columns $\omega_{1},\omega_{2}, ..., \omega_{K}$. Each $\omega_{j}$ is the set of linear combination weights used to create the $j$ th predictive components, thus $Z \Omega_{K}$ is the dimension-reduced version of the original predictor set. Likewise, the predictive coefficient $\theta_{K}$ is now a $K\times 1$ vector rather than $P\times 1$. 

PCR chooses the combination weights $\Omega_{K}$ recursively. The $j^{th}$ linear combination solves

$$\omega_{j} = \text{arg} \max_{\omega} \text{Var} (Z \omega), \text{s.t.} \;
\omega'\omega = 1, \text{Cov}(Z \omega,Z \omega_{l}) = 0, l = 1,2,...,j-1.$$

Intuitively, PCR seeks the $K$ linear combinations of $Z$ that most faithfully mimic the full predictor set. The objective illustrates that the choice of components is not based on the forecasting objective at all. Instead, the emphasis of PCR is on finding components that retain the most possible common variation within the predictor set. The well known solution for PCR computes $\Omega_{K}$ via singular value decomposition of $Z$, and therefore the PCR algorithm is extremely efficient from a computational standpoint.
In contrast to PCR, the PLS objective seeks $K$ linear combinations of $Z$ that have maximal predictive association with the forecast target. The weights used to construct the $j^{th}$ PLS component solve

$$\omega_{j} = \text{arg} \max_{\omega} \text{Cov}^{2} (R,Z \omega), \text{s.t.} \;
\omega'\omega = 1, \text{Cov}(Z \omega,Z \omega_{l}) = 0, l = 1,2,...,j-1.$$

This objective highlights the main distinction between PCR and PLS. PLS is willing to sacrifice how accurately $Z\Omega_{K}$ approximates $Z$ in order to find components with more potent return predictability. The problem of PLS can be efficiently solved using a number of similar routines, the most prominent being the SIMPLS algorithm of de Jong (1993).

Finally, given a solution for $\Omega_{K}$, $\theta_{K}$ is estimated in both PCR and PLS via OLS regression of $R$ on $Z\Omega_{K}$. For both models, $K$ is a hyperparameter that can be determined adaptively from the validation sample.

Then, we will use previous function `PCR()` and `pls()` to create PCA regression and PLS regression. Open the file `Reg.py`, follow the steps to implement function `PCAR()` and `PLAR()`. Test your code here, you should get error on the order of `e-7` or less.

In [ ]:
from Reg import PCAR
np.random.seed(6)
N      = 10
p      = 8
xtrain = np.random.randn(N,p) * 0.01
ytrain = np.random.randn(N,1) * 0.01
mtrain = np.mean(ytrain)
ytrain_demean = ytrain - mtrain

xoos   = np.random.randn(N,p) * 0.01
yoos   = np.random.randn(N,1) * 0.01

xtest  = np.random.randn(N,p) * 0.01
ytest  = np.random.randn(N,1) * 0.01

ne     = 5


r2_oos,r2_is,b,v = PCAR(xtrain,ytrain,ytrain_demean,mtrain,xoos,yoos,xtest,ytest,ne)

expect_roos      = -0.30884144116077
expect_ris       = 0.113068966716010
expect_b         = np.array([[0.0802575,0.01376213,0.0884982,-0.03628063,
                             -0.16151551,0.02924157,0.04834153,-0.0733434]])
expect_v         = np.array([[-0.00605116,-0.01083863,0.01308499,0.0020043,
                               0.03914248,-0.00527651,0.04044305,-0.00278299]])
print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))

In [ ]:
from Reg import PLSR
np.random.seed(6)
N      = 10
p      = 8
xtrain = np.random.randn(N,p) * 0.01
ytrain = np.random.randn(N,1) * 0.01
mtrain = np.mean(ytrain)
ytrain_demean = ytrain - mtrain

xoos   = np.random.randn(N,p) * 0.01
yoos   = np.random.randn(N,1) * 0.01

xtest  = np.random.randn(N,p) * 0.01
ytest  = np.random.randn(N,1) * 0.01

ne     = 5

r2_oos,r2_is,b,v = PLSR(xtrain,ytrain,ytrain_demean,mtrain,xoos,yoos,xtest,ytest,ne)

expect_roos      = -2.64665594438756
expect_ris       = 0.355202101604475
expect_b         = np.array([[0.10038269,-0.38864287,0.22868396,-0.1077111,
                             -0.37620204,-0.05061696,0.54888731,-0.10922797]])
expect_v         = np.array([[-0.04917611,0.22886778,-0.0102955,-0.00796865,
                              -0.21511732,0.01438165,0.16142618,-0.02715686]])
print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))

## Lasso, Ridge and Enet
The simple linear model is bound to fail in the presence of many predictors. When the number of predictors $P$ approaches the number of observations $T$ , the linear model becomes inefficient or even inconsistent. It begins to overfit noise rather than extracting signal. This is particularly troublesome for the problem of return prediction where the signal-to-noise ratio is notoriously low.

Crucial for avoiding overfit is reducing the number of estimated parameters. The most common machine learning device for imposing parameter parsimony is to append a penalty to the objective function in order to favor more parsimonious specifications. This “regularization” of the estimation problem mechanically deteriorates a model’s in-sample performance in hopes that it improves its stability out of sample. This will be the case when penalization manages to reduce the model’s fit of noise, while preserving its signal fit.

The statistical model for our penalized linear model is the same as the simple linear model. That is, it continues to consider only the baseline, untransformed predictors. Penalized methods differ by appending a penalty to the original loss function:

$$\mathcal{L} (\theta;\cdot) = \mathcal{L}(\theta) + \phi(\theta;\cdot)$$

There are several choices for the penalty function $\phi(\theta;\cdot)$. We focus on the popular “elastic net” penalty, which takes the form:

$$ \phi(\theta ; \lambda, \rho) = \lambda ( 1 - \rho) 
\sum_{j=1}^{P} | \theta_{j}| + \frac{1}{2} \lambda \rho 
\sum_{j=1}^{P} \theta_{j}^{2}$$

The elastic net involves two nonnegative hyperparameters, $\lambda$ and $\rho$, and includes two well-known regularizers as special cases. The $\rho = 0$ case corresponds to the lasso and uses an absolute value, or “$l_{1}$,” parameter penalization. The fortunate geometry of the lasso sets coefficients on a subset of covariates to exactly zero. In this sense, the lasso imposes sparsity on the specification and can thus be thought of as a variable *selection* method. The $\rho = 1$ case corresponds to ridge regression, which uses an $l_{2}$ parameter penalization, that draws all coefficient estimates closer to zero but does not impose exact zeros anywhere. In this sense, ridge is a shrinkage method that helps prevent coefficients from becoming unduly large in magnitude. For intermediate values of $\rho$, the elastic net encourages simple models through both shrinkage and selection.

We adaptively optimize the tuning parameters, $\lambda$ and $\rho$, using the validation sample. Our implementation of penalized regression uses the accelerated proximal gradient algorithm and accommodates both least squares and Huber objective functions.


Now, you can implement `Lasso()`, `Ridge()` and `Enet()` to create **Lasso, LassoH, Ridge, RidgeH, Enet and Enet+H** model. Test your code here. You should get error on the order of `e-5` or less.

In [ ]:
from Reg import Lasso

np.random.seed(8)

N      = 10
p      = 8
xtrain = np.random.randn(N,p) * 0.1
ytrain = np.random.randn(N,1) * 0.1
mtrain = np.mean(ytrain)
ytrain_demean = ytrain - mtrain

XX     = xtrain.T.dot(xtrain) 
XY     = xtrain.T.dot(ytrain_demean)

xoos   = np.random.randn(N,p)  * 0.1
yoos   = np.random.randn(N,1)  * 0.1

xtest  = np.random.randn(N,p) * 0.1
ytest  = np.random.randn(N,1) * 0.1 

tol    = 1e-3
L      = 1000
alpha  = 1
lamv   = [-6,-5,-4]
mu     = 0.001


r2_oos,r2_is,b,v,r2_oos_H,r2_is_H,b_H,v_H = Lasso(XX,XY,xtrain,ytrain,\
                                                      ytrain_demean,mtrain,\
                                                      xoos,yoos,xtest,ytest,\
                                                      mu,tol,L,alpha,lamv)
expect_roos   =  0.017764443311936495
expect_ris    =  0.11727752632364752
expect_b      = [-0.01298148,0.01059046,-0.00955958,-0.02074591,-0.00275158,
                 -0.01082944,0.01415878,0.00520742]
expect_v      = [0.01598016,0.01074124,0.00847576,0.04256298,0.00084163,
                 0.01224452,0.0194681,0.00231169]
expect_roos_H =  0.01776444
expect_ris_H  =  0.11727753
expect_b_H    = [-0.01298148,0.01059046,-0.00955958,-0.02074591,-0.00275158,
                 -0.01082944,0.01415878,0.00520742]
expect_v_H    = [0.01598016,0.01074124,0.00847576,0.04256298,0.00084163,
                 0.01224452,0.0194681,0.00231169]
print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))
print(rel_error(r2_oos_H,expect_roos_H))
print(rel_error(r2_is_H,expect_ris_H))
print(rel_error(b_H,expect_b_H))
print(rel_error(v_H,expect_v_H))

In [ ]:
from Reg import ridge

np.random.seed(9)

N      = 10
p      = 8
xtrain = np.random.randn(N,p) 
ytrain = np.random.randn(N,1) 
mtrain = np.mean(ytrain)
ytrain_demean = ytrain - mtrain

XX     = xtrain.T.dot(xtrain) 
XY     = xtrain.T.dot(ytrain_demean)

xoos   = np.random.randn(N,p)  
yoos   = np.random.randn(N,1)  

xtest  = np.random.randn(N,p) 
ytest  = np.random.randn(N,1)  

tol    = 1e-3
L      = 1000
alpha  = 1
lamv   = [-7,-6,-5]
mu     = 0.01


r2_oos,r2_is,b,v,r2_oos_H,r2_is_H,b_H,v_H = ridge(XX,XY,xtrain,ytrain,\
                                                      ytrain_demean,mtrain,\
                                                      xoos,yoos,xtest,ytest,\
                                                      mu,tol,L,alpha,lamv)

expect_roos   = -0.13576377873651002
expect_ris    = 0.767011957407248
expect_b      = [0.00785283,0.50855187,-0.22865912,0.0178831,
                -0.25993783,-0.00427208,0.20439858,0.24239826]
expect_v      = [-4.88042803e-04,4.47277473e-01,  4.30400931e-02,3.24499783e-04,
                  1.99495639e-02,6.20250496e-04,2.15964606e-01,1.12171943e-01]
expect_roos_H = -0.13579605
expect_ris_H  = 0.76702826
expect_b_H    = [ 0.00784691,0.50858005,-0.22868178,0.01788824,
                 -0.25994479,-0.00427235,0.20441274,0.24243719]
expect_v_H    = [-4.87552052e-04,4.47312809e-01,4.30404119e-02,3.23845178e-04,
                  1.99377721e-02,6.20430699e-04,2.15969606e-01,1.12177408e-01]

print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))
print(rel_error(r2_oos_H,expect_roos_H))
print(rel_error(r2_is_H,expect_ris_H))
print(rel_error(b_H,expect_b_H))
print(rel_error(v_H,expect_v_H))

In [ ]:
from Reg import Enet

np.random.seed(10)

N      = 10
p      = 8
xtrain = np.random.randn(N,p) 
ytrain = np.random.randn(N,1) 
mtrain = np.mean(ytrain)
ytrain_demean = ytrain - mtrain

XX     = xtrain.T.dot(xtrain) 
XY     = xtrain.T.dot(ytrain_demean)

xoos   = np.random.randn(N,p)  
yoos   = np.random.randn(N,1)  

xtest  = np.random.randn(N,p) 
ytest  = np.random.randn(N,1)  

tol    = 1e-3
L      = 1000
alpha  = 1
lamv   = [-7,-6,-5]
mu     = 0.01


r2_oos,r2_is,b,v,r2_oos_H,r2_is_H,b_H,v_H = Enet(XX,XY,xtrain,ytrain,\
                                                      ytrain_demean,mtrain,\
                                                      xoos,yoos,xtest,ytest,\
                                                      mu,tol,L,alpha,lamv)


expect_roos   = -0.9102052662776483
expect_ris    = 0.5164359379277887
expect_b      = [0.29423895,-0.04053108,-0.10959231,0.05154946,
                 0.60408725,0.31032992,0.53226383,0.17623848]
expect_v      = [0.03960828,-0.00078205,0.00219732,0.00710816,
                 0.202848,0.06113573,0.11937089,0.04743008]
expect_roos_H = -0.9101137
expect_ris_H  = 0.51644045
expect_b_H    = [0.29419352,-0.04055566,-0.10958457,0.05150143,
                 0.60408996,0.31030545,0.53225886,0.17629508]
expect_v_H    = [0.03962512,-0.00078528,0.00220263,0.00710675,
                 0.20286353,0.06115181,0.11936435,0.04742936]

print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))
print(rel_error(r2_oos_H,expect_roos_H))
print(rel_error(r2_is_H,expect_ris_H))
print(rel_error(b_H,expect_b_H))
print(rel_error(v_H,expect_v_H))

## Oracle Linear Model
In the oracle linear model, we make a small change in convariant matrix. Open the file `Reg.py` and implement the function `Oracle()`. Test your code here. You should get error of magnitude `e-9` or less.

In [ ]:
from Reg import Oracle

np.random.seed(11)

N      = 10
p      = 8
xtrain = np.random.randn(N,p) 
ytrain = np.random.randn(N,1) 
mtrain = np.mean(ytrain)

xoos   = np.random.randn(N,p)  
yoos   = np.random.randn(N,1)

nump   = 5

mo     = 1
r2_oos,r2_is = Oracle(mo,nump,xtrain,ytrain,mtrain,xoos,yoos)

expect_roos  = -1.51856578
expect_ris   = 0.30796397
print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))


## Group Lasso 

Linear models are popular in practice, in part because they can be thought of as a first-order approximation to the data generating process. When the “true” model is complex and nonlinear, restricting the functional form to be linear introduces approximation error due to model misspecification. Let $g^{\star}(z_{i,t})$ denote the true model and $g(z_{i,t};\theta)$ the functional form specified by the econometrician. And let $g(z_{i,t};\hat{\theta})$ and $\hat{r}_{i,t+1}$ denote the fitted model and its
ensuing return forecast. We can decompose a model’s forecast error as:


$$ \begin{matrix}
r_{i,t+1} - \hat{r}_{i,t+1} = &\underbrace{g^{\star}(z_{i,t})-g(z_{i,t};\theta)} \;+ &\underbrace{g(z_{i,t};\theta) - g(z_{i,t};\hat{\theta})} \;+&\underbrace{\epsilon_{i,t}}
\\ &\text{approximation error} & \text{estimation error} &\text{intrinstic error} 
\end{matrix}$$

Intrinsic error is irreducible; it is the genuinely unpredictable component of returns associated with news arrival and other sources of randomness in financial markets. Estimation error, which arises due to sampling variation, is determined by the data. It is potentially reducible by adding new observations, though this may not be under the econometrician’s control. Approximation error is directly controlled by the econometrician and is potentially reducible by incorporating more flexible specifications that improve the model’s ability to approximate the true model. But additional flexibility raises the risk of overfitting and destabilizing the model out of sample. In this and the following subsections, we introduce nonparametric models of $g(\cdot)$ with increasing degrees of flexibility, each complemented by regularization methods to mitigate overfit.

The first and most straightforward nonparametric approach that we consider is the generalized linear model. It introduces nonlinear transformations of the original predictors as new additive terms in an otherwise linear model. Generalized linear models are thus the closest nonlinear counterparts to the linear approaches in previous section. 

The model we study adapts the simple linear form by adding a $K$-term spline series expansion of the predictors

$$ g(z;\theta,p(\cdot)) = \sum_{j=1}^{P} p(z_{j})' \theta_{j}$$

where $p(\cdot)=(p_1(\cdot),p_2(\cdot),...,p_{K}(\cdot))'$ is a vector of basis functions, and the parameters are now a $K\times N$ matrix $\theta =(\theta_1,\theta_2,...,\theta_{N})$. There are many potential choices for spline functions. We adopt a spline series of order two: $\left(1, z, (z-c_1)^{2}, (z-c_2)^{2}, ..., (z-c_{K-2}\;)^{2}\; \right)$, where $c_1, c_2, …c_{K−2}$ are knots.

Because higher-order terms enter additively, forecasting with the generalized linear model can be approached with the same estimation tools as in simple linear model. In particular, our analysis uses a least squares objective function, both with and without the Huber robustness modification. Because series expansion quickly multiplies the number of model parameters, we use penalization to control degrees of freedom. Our choice of penalization function is specialized for the spline expansion setting and is known as the group lasso. It takes the form

$$\phi(\theta ; \lambda,K) = \lambda \sum_{j=1}^{P} \left(
\sum_{k=1}^{K} \theta_{j,k}^{2}
\right)^{1/2}$$

As its name suggests, the group lasso selects either all $K$ spline terms associated with a given characteristic or none of them. We embed this penalty in the general objective of penalized model. Group lasso accommodates either least squares or robust Huber objective, and it uses the same accelerated proximal gradient descent as the elastic net. It has two tuning parameters, $\lambda$ and $K$.

Open the file `Reg.py` and implement the function `Group_lasso()`. Test your code here and you should get an error of magnitude `e-7` or less.

In [ ]:
from Reg import Group_lasso

np.random.seed(12)

N      = 10
p      = 2
xtrain = np.random.randn(N,p) 
ytrain = np.random.randn(N,1) 
mtrain = np.mean(ytrain)
ytrain_demean = ytrain - mtrain

xoos   = np.random.randn(N,p)  
yoos   = np.random.randn(N,1)

xtest  = np.random.randn(N,p)  
ytest  = np.random.randn(N,1)

kn     = 3

mu     = 0.001
lamv   = [-1,-2]
tol    = 1e-5

r2_oos,r2_is,b,v,r2_oos_H,r2_is_H,b_H,v_H = Group_lasso(kn,xtrain,ytrain,\
                                                        ytrain_demean,mtrain,\
                                                        xoos,yoos,xtest,ytest,mu,\
                                                        tol,lamv)


expect_roos   = -13.774661580655398
expect_ris    = 0.7520909205110202
expect_b      = [-1.3814082,5.49855818,-5.45226757,1.03838396,0.01786894,
                  0.05652455,0.05413286,0.01199471]
expect_v      = [1.55230372e+00,2.45922982e+01,2.41808615e+01,8.77280318e-01,
                 3.18114214e-04,3.24397984e-03,2.96481549e-03,1.44667755e-04]
expect_roos_H = -13.76682029
expect_ris_H  = 0.75202276
expect_b_H    = [-1.38108829,5.49731366,-5.45099348,1.03819251,
                  0.017472,0.05524136,0.05289458,0.01169055]
expect_v_H    = [1.55657585e+00,2.45640198e+01,2.41817551e+01,8.75582129e-01,
                 3.88133765e-04,3.39374814e-03,3.11249465e-03,1.96395816e-04]

print(rel_error(r2_oos,expect_roos))
print(rel_error(r2_is,expect_ris))
print(rel_error(b,expect_b))
print(rel_error(v,expect_v))
print(rel_error(r2_oos_H,expect_roos_H))
print(rel_error(r2_is_H,expect_ris_H))
print(rel_error(b_H,expect_b_H))
print(rel_error(v_H,expect_v_H))


# Part 3: Start Monte Carlo Simulation

Now, you are ready for the MC simulation. In this section, you only need to provide some parameters, such as the times for MC simulation, P, horizon and model type.

Start your simulation now! 

This section is cited of paper [Empirical Asset Pricing via Machine Learning](https://dachxiu.chicagobooth.edu/download/ML.pdf).

In [ ]:
from Reg import MC_simulation
MC      = [1,10]
datanum = 100
horizon = [1]
mo      = [1]
MC_simulation(MC,datanum,horizon,mo)